In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import torch  
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


c:\Users\markm\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Neural Net Class and Training Functions
Define Class and functions

In [2]:
def train_model(nn_model, X_train, y_train, X_eval, y_eval, lr=1e-4, max_iter=50, batch_size=32, print_n=10):
    '''
    Trains neural network model on X_train, y_train data.

    Parameters
    ----------
    X_train: np.array
        matrix of training data features
    y_train: np.array
        vector of training data labels
    lr: float
        learning_rate for training
    max_iter: int
        maximum number of iterations to train for
    batch_size: int
        batch size to use when training w/ SGD
    print_n: int
        print training progress every print_n steps

    Returns
    ----------
    nn_model: torch.nn.Module
        trained neural network model
    '''
    # convert to tensors (for Pytorch)
    X_train_tensor = torch.tensor(X_train)
    y_train_tensor = torch.tensor(y_train)
    X_test_tensor = torch.tensor(X_eval)
    y_test_tensor = torch.tensor(y_eval)
    # intialize neural network
    n_samples, n_features = X_train_tensor.shape
    nn_model.train()  # put model in train mode
    # initialize mse loss function
    mse_loss = torch.nn.MSELoss()
    # train with (mini-batch) SGD; initialize optimizer
    #opt = torch.optim.SGD(nn_model.parameters(), lr=lr)
    opt = torch.optim.SGD(nn_model.parameters(), lr=lr,  momentum=0.9)
    losses_test = []
    # save losses across all iterations
    losses = []
    for it in range(max_iter):
        loss_train_sum = 0
        loss_test_sum = 0 
        # loop through data in batches
        for batch_start in range(0, n_samples, batch_size):
            # reset gradients to zero
            opt.zero_grad()
            # form batch
            X_batch = X_train_tensor[batch_start:batch_start+batch_size]
            y_batch = y_train_tensor[batch_start:batch_start+batch_size]
            X_batch_test = X_test_tensor[batch_start:batch_start+batch_size]
            y_batch_test = y_test_tensor[batch_start:batch_start+batch_size]
            # pass batch through neural net to get prediction
            y_pred = nn_model(X_batch.float())
            y_pred = y_pred.unsqueeze(1)
            y_pred_test = nn_model(X_batch_test.float())
            y_pred_test = y_pred_test.unsqueeze(1)
            # compute MSE loss
            loss = mse_loss(y_pred, y_batch[:, None].float())
            loss_train_sum += loss.detach().numpy()
            loss_test = mse_loss(y_pred_test, y_batch_test[:, None].float())
            loss_test_sum += loss_test.detach().numpy()
            # back-propagate loss
            loss.backward()
            # update model parameters based on backpropogated gradients - clip values to avoid exploding gradients
            torch.nn.utils.clip_grad_value_(nn_model.parameters(), clip_value=1.5)
            opt.step()
        losses.append(loss_train_sum)
        losses_test.append(loss_test_sum)
        
        if it % print_n == 0:
            print(f"Mean Train MSE at step {it}: {loss_train_sum}")

        
        
    return nn_model, losses_test

def evaluate_model(nn_model, X_eval, y_eval, batch_size=32):
    '''
    Evaluates trained neural network model on X_eval, y_eval data.

    Parameters
    ----------
    nn_model: torch.nn.Module
        trained neural network model
    X_eval: np.array
        matrix of training data features
    y_eval: np.array
        vector of training data labels
    batch_size: int
        batch size to looping over dataset to generate predictions

    Returns
    ----------
    mse: float
        MSE of trained model on X_eval, y_eval data
    '''
    # initialize mse loss function
    mse_loss = torch.nn.MSELoss()
    # convert to tensors (for Pytorch)
    X_eval_tensor = torch.tensor(X_eval)
    y_eval_tensor = torch.tensor(y_eval)
    n_samples = X_eval_tensor.shape[0]
    nn_model.eval() # put in eval mode
    # loop over data and generate predictions
    preds = []
    for batch_start in range(0, n_samples, batch_size):
        # form batch
        X_batch = X_eval_tensor[batch_start:batch_start+batch_size]
        y_batch = y_eval_tensor[batch_start:batch_start+batch_size]
        with torch.no_grad():  # no need to compute gradients during evaluation
            # pass batch through neural net to get prediction
            y_pred = nn_model(X_batch.float())
            y_pred = y_pred.unsqueeze(1)
            preds.append(y_pred)
    # compute MSE across all samples
    all_preds = torch.cat(preds)
    loss = mse_loss(all_preds, y_eval_tensor[:, None].float()).item()
    y_pred = torch.reshape(all_preds, (y_eval.shape[0], 1))

    r_square = r2_score(y_eval, y_pred)
    return loss, r_square

class NN(nn.Module):
    '''
    Class for fully connected neural net.
    '''
    def __init__(self, input_dim, hidden_dim):
        '''
        Parameters
        ----------
        input_dim: int
            input dimension (i.e., # of features in each example passed to the network)
        hidden_dim: int
            number of nodes in hidden layer
        '''
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.layers = nn.Sequential(
            # Network should have a single hidden layer
            # Apply ReLU activation in between the hidden layer and output node
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.layers(x)
        return x


class NN_configureable(nn.Module):
    '''
    Class for fully connected neural net.
    '''
    def __init__(self, input_dim, hidden_dim, hidden_layers):
        '''
        Parameters
        ----------
        input_dim: int
            input dimension (i.e., # of features in each example passed to the network)
        hidden_dim: int
            number of nodes in hidden layer
        '''
        super().__init__()
        #self.input_dim = input_dim
        self.hidden_layers = hidden_layers
        self.layers = nn.ModuleDict()

        # Define input layer
        self.layers["input"] = nn.Linear(in_features = input_dim, out_features = hidden_dim)
        # Define hidden layers
        for i in range(self.hidden_layers):
            self.layers[f"hidden_{i}"] = nn.Linear(in_features = hidden_dim, out_features = hidden_dim)
        # Define output layer
        self.layers["output"] = nn.Linear(in_features = hidden_dim, out_features = 1)

    def forward(self, x):
        x = self.layers["input"](x)
        for i in range(self.hidden_layers):
            x = F.relu(self.layers[f"hidden_{i}"](x))

        return self.layers["output"](x)        
        

# Import Data
Get data from different sources before combining
* Cleaned up EV data: TX_WA_CO_NY.csv
* Average EV price and new car data over time: Avg_EV_Price.csv
* Census data (pop, household income, zipcode): census.csv 

In [7]:
def main():   
    # Import data
    X = pd.read_csv('./Data/df_X_county.csv')
    #X['constant'] = 1
    y = pd.read_csv('./Data/df_y_county.csv')

    # check if any nan values
    nan_row_X = X[X.isna().any(axis=1)]
    #print(nan_row_X)
    nan_row_y = y[y.isna().any(axis=1)]
    #print(nan_row_y)

    X = X.to_numpy()
    y = y.to_numpy()

    # only predict PHEV
    y = y[:,0]

    # split train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # standardize X
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # train NN model to predict EV registration using train data
    X_train_tensor = torch.tensor(X_train)
    n_samples, n_features = X_train_tensor.shape
    
    # Input (manually) optimal hyper-paramater results from RayTune grid search
    best_hidden_dim = 8
    best_hidden_layer = 4
    best_batch_size = 64
    best_lr = 0.0004697091486004313
    best_max_iter = 50
    nn_model = NN_configureable(input_dim=n_features, hidden_dim=best_hidden_dim, hidden_layers=best_hidden_layer)
    nn_model_result = train_model(nn_model, X_train, y_train, X_test, y_test, lr=best_lr, max_iter=best_max_iter, batch_size=best_batch_size)
    
    train_mse = evaluate_model(nn_model_result[0], X_train, y_train, batch_size=best_batch_size)
    test_mse = evaluate_model(nn_model_result[0], X_test, y_test, batch_size=best_batch_size)
    
    # Save the trained model to the models folder
    joblib.dump(nn_model, 'Models/model_nn.joblib')
    
    print(f"Train MSE for model, dim = {best_hidden_dim} is: {train_mse} Train R-squared is: {train_mse[1]}")
    print(f"Test MSE for model, dim = {best_hidden_dim} is: {test_mse[0]} Test R-squared is: {test_mse[1]}")

    '''
    # plot 3 models test errors
    print(nn_model_result[1])
    plt.plot(range(len(nn_model_result[1])), nn_model_result[1])
    # axis labels
    plt.xlabel('Iteration Step')
    plt.ylabel('Test Error')
    plt.title("Model 1 - Hidden Layer - ReLU")
    plt.legend()
    plt.figure(figsize=(10,6))
    plt.show()
    '''

In [8]:
if __name__ == '__main__':
    main()

c:\Users\markm\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\markm\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([42, 1])) that is different to the input size (torch.Size([42, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\markm\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([0, 1])) that is different to the input size (torch.Size([0, 1, 1])). This will like

Mean Train MSE at step 0: 91536529.78125
Mean Train MSE at step 10: 84340066.9921875
Mean Train MSE at step 20: 84356850.3203125
Mean Train MSE at step 30: 84364758.9453125
Mean Train MSE at step 40: 84379844.890625


c:\Users\markm\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([8871, 1])) that is different to the input size (torch.Size([8871, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\markm\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2218, 1])) that is different to the input size (torch.Size([2218, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Train MSE for model, dim = 8 is: (606754.5625, -0.00424957359658773) Train R-squared is: -0.00424957359658773
Test MSE for model, dim = 8 is: 595430.625 Test R-squared is: -0.004352720448543623
